# [Burrows-Wheeler Transform (BWT)](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Burrows+M%2C+Wheeler+DJ%3A+A+Block+Sorting+Lossless+Data+Compression+Algorithm.&btnG=)

* BWT is an algorithm that inputs a string and outputs:
  
  1. A different string with the same symbols but with longer runs, and therefore potentially more compressible.
     The lengths of the runs are proportional to the correlation
     among symbols and the length of the input.
  2. An index.
  
* The inverse transform, using the output of the
  forward transform, recover the original string.
  
* Used in [`bzip2`](https://en.wikipedia.org/wiki/Bzip2).

## Encoder

Let $B$ the block-size in symbols:

1. While the input is not exhausted:
    1. Read $B$ symbols in `S`.
    2. (`L`, `I`) = BWT(`S`).
    3. Output `L` (the output block with longer runs) and `I` (an index of a symbol in `L`).

## Decoder

1. While input pairs (`L`, `I`):
    1. `S` = iBWT(`L`, `I`).
    2. Output `S`.

## Forward BWT

1. Input `S`, the sequence of `B` symbols.
    ```
    S = "abraca."
    ```
    
2. Compute a matrix `M'` with all possible cyclic rotations of `S`.

    ```
    M' = ["abraca.",
          "braca.a",
          "raca.ab",
          "aca.abr",
          "ca.abra",
          "a.abrac",
          ".abraca"]
    ```

3. Sort lexicographically `M'` go generate `M`.
    ```
    M = [".abraca",
         "a.abrac",
         "abraca.",
         "aca.abr",
         "braca.a",
         "ca.abra",
         "raca.ab"]
    ```

3. Let `I` the index of `S` in `M`.
    ```
    I = 2
    ```

4. Let `L` the column `B`-1 of `M`.
    ```
    L = "ac.raab"
    ```

2. Output `I` and `L`.

## Inverse BWT

The backward transform regenerates the `I`-th row of `M`. Here there is an example:

1. Input `I` and `L`, the output of a BWT applied to a string `S` of length `B`.

2. The first `F` and the last `L` columns of `M` are available taking into consideration that `F=sorted(L)`.
    ```
    F23456L
    .     a
    a     c
    a     .
    a     r
    b     a
    c     a
    r     b
    ```
    
3. Notice that for a particular symbol in `L`, the corresponding symbol in `F` follow it in `S`
   (for example, `r` follows `b` in `abraca.`). Therefore, we have found all pairs of `S` by
   taking pairs of `LF`.
    ```
    a.
    ca
    .a
    ra
    ab
    ac
    br
    ```
   Which sorted:
    ```
    .a
    a.
    ab
    ac
    br
    ca
    ra
    ```
   become the first two columns of `M`.

4. Repeat the process until getting the rest of the columns of `M` (here only a few are shown):

    ```
    F23456L
    .a    a
    a.    c
    ab    .
    ac    r
    br    a
    ca    a
    ra    b
    ```
   Now, for a particular symbol in `L`, the corresponding pair in columns `F`
   and `2` follows it in `S` (for example, pair `br` follows symbol `a` in `abraca.`).
   So, we can find all triples of `S` by tacking triples of `LF2`:
    ```
    a.a         .ab
    ac.         a.a
    .ab  sort   abr
    rac ------> aca
    abr         bra
    aca         ca.
    bra         rac
    ```
   to have the partial reconstruction of `M`:
    ```
    F23456L
    .ab   a
    a.a   c
    abr   . <- I
    aca   a
    bra   a
    ca.   a
    rac   b
    ```

In an optimized implementation of the BWT, only the row `I` is generated.

### Lab

In [1]:
# https://gist.github.com/dmckean/9723bc06254809e9068f

def BWT(S):
    
    if __debug__:
        print('Original string:')
        print("S =", S)
        
    n = len(S)

    N = [S[i:n]+S[0:i] for i in range(n)]

    if __debug__:
        print('')
        print('Permutations matrix:')
        counter = 0
        for i in N:
            print(i, counter)
            counter += 1

    # 1. Matrix of all possible rotations (cyclid shifts) of 's'.
    M = sorted(N)
    
    if __debug__:
        print('')
        print('Sorted matrix:')
        counter = 0
        for i in M:
            print(i, counter)
            counter += 1

    # 2. I = the index of 's' in 'M'.
    I = M.index(S) 
    
    # 3. L = the last column of 'M'.
    L = ''.join([q[-1] for q in M])

    if __debug__:
        print('')
        print('M\' = M rotated one character to the right')
        Mp = []
        for i in range(n):
            Mp.append(M[i][n-1:n]+M[i][0:n-1])
        for i in range(n):
            print(Mp[i])
    
    return (I, L)

from operator import itemgetter

def iBWT(I, L):
    n = len(L)
    
    # 1. Compute correspondence between the rows of M and M'.
    X = sorted([(i, x) for i, x in enumerate(L)], key=itemgetter(1))
    T = [None for i in range(n)]
    for i, y in enumerate(X):
        j, _ = y
        T[j] = i
        
    if __debug__:
        print("T = Positions of rows of M\' in M:", T)

    # 2. for i in range(n): S[n-1-i] = L[T^i[I]]
    # where T^0[x]=x and T^{i+1}[x] = T[T^i[x]].
    Tx = [I]
    for i in range(1, n):
        Tx.append(T[Tx[i-1]])
    if __debug__:
        print("Tx = Positions in L of output symbols (reversed) =", Tx)
    S = [L[i] for i in Tx]
    S.reverse()
    return ''.join(S)

print("ENCODING")
I, L = BWT('abraca.')
print("")
print ("I = {}, L = {}\n".format(I, L))
print("DECODING")
Sp = iBWT(I, L)
print ("S = {}".format(Sp))

print("")

print("ENCODING")
I, L = BWT('abababababababababababababababababababa')
print("")
print ("I = {}, L = {}\n".format(I, L))
print("DECODING")
Sp = iBWT(I, L)
print ("S = {}".format(Sp))

print("")

print("ENCODING (run are formed by the symbol that define a context)")
I, L = BWT('abacadaeaf')
print("")
print ("I = {}, L = {}\n".format(I, L))
print("DECODING")
Sp = iBWT(I, L)
print ("S = {}".format(Sp))

print("")

print("ENCODING (or by the symbol that goes after one or more contexts)")
I, L = BWT('bacadaeafa')
print("")
print ("I = {}, L = {}\n".format(I, L))
print("DECODING")
Sp = iBWT(I, L)
print ("S = {}".format(Sp))

print("")

print("ENCODING ( see https://link.springer.com/book/10.1007/978-0-387-78909-5 )")
I, L = BWT('aardvark$')
print("")
print ("I = {}, L = {}\n".format(I, L))
print("DECODING")
Sp = iBWT(I, L)
print ("S = {}".format(Sp))


ENCODING
Original string:
S = abraca.

Permutations matrix:
abraca. 0
braca.a 1
raca.ab 2
aca.abr 3
ca.abra 4
a.abrac 5
.abraca 6

Sorted matrix:
.abraca 0
a.abrac 1
abraca. 2
aca.abr 3
braca.a 4
ca.abra 5
raca.ab 6

M' = M rotated one character to the right
a.abrac
ca.abra
.abraca
raca.ab
abraca.
aca.abr
braca.a

I = 2, L = ac.raab

DECODING
T = Positions of rows of M' in M: [1, 5, 0, 6, 2, 3, 4]
Tx = Positions in L of output symbols (reversed) = [2, 0, 1, 5, 3, 6, 4]
S = abraca.

ENCODING
Original string:
S = abababababababababababababababababababa

Permutations matrix:
abababababababababababababababababababa 0
bababababababababababababababababababaa 1
ababababababababababababababababababaab 2
babababababababababababababababababaaba 3
abababababababababababababababababaabab 4
bababababababababababababababababaababa 5
ababababababababababababababababaababab 6
babababababababababababababababaabababa 7
abababababababababababababababaabababab 8
bababababababababababababababaababababa 9
a